In [1]:
%load_ext autoreload
%autoreload 2

import os
if not 'DISPLAY' in os.environ:
    os.environ['DISPLAY'] = ':1'

import numpy as np
from skimage import io
from dm_control.suite import common

from genome_synth import Genesis
from findtarget import FindTarget, Physics
from dm_control import mujoco
from dm_control.rl import control
from dm_control.rl.control import PhysicsError

from mjremote import mjremote
import time

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from xml.etree import ElementTree as ET

class DummyGen(Genesis):
    def override_from_xml(self, xml_str):
        self.joints = set()
        spec = ET.fromstring(xml_str)
        for motor in spec.iter('motor'):
            if 'joint' in motor.attrib:
                self.joints.add(motor.attrib['joint'])


In [3]:
from deap import base, creator, gp, tools

gen = DummyGen()

m = mjremote()
print('Connect:', m.connect())

xml_name = './starfish.xml'

if not os.path.exists('./tmp'):
    os.mkdir('./tmp')

with open(xml_name) as starfishf:
    starfish_str = starfishf.read()
    
def capture():
    imnames = set()
    picidx = 0
    _DEFAULT_TIME_LIMIT = 10
    _CONTROL_TIMESTEP = .04
    display_stride = 1 / .04 // 24
    gen.override_from_xml(starfish_str)
    genesis_physics = Physics.from_xml_string(common.read_model(os.path.join(os.getcwd(), xml_name)), 
                                              common.ASSETS)
    genesis_physics.set_genesis(gen)
    genesis_task = FindTarget()
    genesis_env = control.Environment(genesis_physics, 
                                     genesis_task,
                                     control_timestep=_CONTROL_TIMESTEP,
                                     time_limit=_DEFAULT_TIME_LIMIT)
    action_spec = genesis_env.action_spec()
    observation_spec = genesis_env.observation_spec()
    print(action_spec, observation_spec)
    time_step = genesis_env.reset()
    curtime = 0.0
    #top_view = genesis_env.physics.render(480, 480, camera_id='tracking_top')
    #side_view = genesis_env.physics.render(480, 480, camera_id='arm_eye')
    #plt.imshow(np.concatenate((top_view, side_view), axis=1))
    #plt.pause(0.5)
    #plt.show()
    did_except = False
    
    b = bytearray(3*m.width*m.height)
    
    while(not time_step.last()):
        try:
            action = np.sin(np.linspace(curtime, curtime + 3.14159, num=action_spec.shape[0]))
            time_step = genesis_env.step(action)
            m.setqpos(genesis_env.physics.data.qpos[:])
            m.getimage(b)
            saveimg = np.frombuffer(b, dtype=np.uint8).reshape((m.height, m.width, 3))
            savename = "./tmp/starfish_{0:04}.jpg".format(picidx)
            picidx += 1
            imnames.add(savename)
            curtime += _CONTROL_TIMESTEP
            #top_view = genesis_env.physics.render(480, 480, camera_id='tracking_top')
            #side_view = genesis_env.physics.render(480, 480, camera_id='arm_eye')
            #plt.imshow(np.concatenate((top_view, side_view), axis=1))
            #plt.pause(0.5)
            #io.imsave(savename, np.concatenate((top_view, side_view), axis=1))
            io.imsave(savename, saveimg)
        except PhysicsError:
            print('except')
            did_except = True
            break
    if not did_except and os.path.isfile('starfish.mp4'):
        os.remove('starfish.mp4')
    if not did_except:
        !!ffmpeg -f image2 -pattern_type sequence -i "./tmp/starfish_%4d.jpg" -qscale:v 0 starfish.mp4
    for name in imnames:
        os.remove(name)
    print("done")
capture()

Connect: (31, 0, 2, 640, 480)
BoundedArraySpec(shape=(24,), dtype=dtype('float64'), name=None, minimum=[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.], maximum=[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]) OrderedDict([('joint_angles', ArraySpec(shape=(24,), dtype=dtype('float64'), name=None)), ('target_offset', ArraySpec(shape=(3,), dtype=dtype('float64'), name=None)), ('vertical_orient', ArraySpec(shape=(3,), dtype=dtype('float64'), name=None)), ('velocity', ArraySpec(shape=(30,), dtype=dtype('float64'), name=None))])
1
24
done


In [4]:
starfish_arm = lambda path_so_far, depth: {
    'body': {
        'name': path_so_far + '.' + 'arm_segment' ,
        'pos':  '0.1 0.0 0.0',
        'children': [
            {'joint': {'axis': '1 0 0'}}
        ]
    }
} if depth >= 0 else None

starfish_dict = {
    'body': {'name': 'root', 'childclass': 'starfish', pos: '0 0 1'},
    
}

def dict_2_creature(num_arms, template_xml='starfish.xml'):
    joints = set()
    actuator = set()
    contact_excludes = set()
    
    

NameError: name 'pos' is not defined